In [2]:
!pip install faker


   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
    --------------------------------------- 0.0/2.0 MB 131.3 kB/s eta 0:00:15
    --------------------------------------- 0.0/2.0 MB 131.3 kB/s eta 0:00:15
    --------------------------------------- 0.0/2.0 MB 131.3 kB/s eta 0:00:15
    --------------------------------------- 0.0/2.0 MB 109.3 kB/s eta 0:00:18
   - -------------------------------------- 0.1/2.0 MB 148.8 kB/s eta 0:00:13
   - -------------------------------------- 0.1/2.0 MB 148.8 kB/s eta 0:00:13
   - -------------------------------------- 0.1/2.0 MB 151.3 kB/s eta 0:00:13
   - -------------------------------------- 0.1/2.0 MB 174.7 kB/s eta 0:00:11
   - --------------------------

In [3]:
import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime, timedelta
import random
import os

fake = Faker()
FOLDER = "retail-pricing-analysis"
os.makedirs(os.path.join(FOLDER, "dataset"), exist_ok=True)
os.makedirs(os.path.join(FOLDER, "output"), exist_ok=True)
os.makedirs(os.path.join(FOLDER, "notebooks"), exist_ok=True)
os.makedirs(os.path.join(FOLDER, "dashboard"), exist_ok=True)

# ------------- Generate synthetic data -------------
n_rows = 5000
start_date = datetime(2023,1,1)
date_list = [start_date + timedelta(days=int(x)) for x in np.random.randint(0, 730, size=n_rows)]

categories = ["Electronics","Automotive","Grocery","Apparel"]
regions = ["North","South","East","West"]
customer_segments = ["Retail","Wholesale"]

products = [
    ("P100","Wireless Headphones","Electronics"),
    ("P101","Car Battery","Automotive"),
    ("P102","Organic Rice","Grocery"),
    ("P103","T-Shirt","Apparel"),
    ("P104","Bluetooth Speaker","Electronics"),
    ("P105","Brake Pads","Automotive"),
    ("P106","Olive Oil","Grocery"),
    ("P107","Jeans","Apparel")
]

rows = []
for i in range(n_rows):
    pid, pname, cat = random.choice(products)
    base_price = round(abs(np.random.normal(loc=100 if cat=="Grocery" else 200, scale=50)),2)
    discount_pct = round(np.random.choice([0,5,10,15,20,25,30], p=[0.15,0.15,0.2,0.2,0.15,0.1,0.05]),2)
    selling_price = round(base_price * (1 - discount_pct/100),2)
    competitor_price = round(selling_price * np.random.uniform(0.9,1.15),2)
    units_sold = max(1, int(np.random.poisson(5 if cat!="Electronics" else 2)))
    revenue = round(selling_price * units_sold,2)
    region = random.choice(regions)
    segment = random.choices(customer_segments, weights=[0.8,0.2])[0]
    stock = max(0, int(np.random.normal(200,80)))
    cost = round(base_price * np.random.uniform(0.55,0.75),2)
    rows.append({
        "Date": date_list[i].date(),
        "Product_ID": pid,
        "Product_Name": pname,
        "Category": cat,
        "Base_Price": base_price,
        "Discount_%": discount_pct,
        "Selling_Price": selling_price,
        "Competitor_Price": competitor_price,
        "Units_Sold": units_sold,
        "Revenue": revenue,
        "Region": region,
        "Customer_Segment": segment,
        "Stock_Level": stock,
        "Cost": cost
    })

df = pd.DataFrame(rows)

# ------------- Introduce some missing values (realistic) -------------
for col in ["Selling_Price", "Competitor_Price"]:
    df.loc[df.sample(frac=0.02, random_state=1).index, col] = np.nan
df.loc[df.sample(frac=0.015, random_state=2).index, "Units_Sold"] = np.nan

# ------------- Cleaning & Feature Engineering -------------
# Fill missing Units_Sold with median by Category
df['Units_Sold'] = df.groupby('Category')['Units_Sold'].transform(lambda x: x.fillna(round(x.median())))

# If Selling_Price missing, compute from Base_Price & Discount_%
df['Selling_Price'] = df['Selling_Price'].fillna((df['Base_Price'] * (1 - df['Discount_%']/100)).round(2))

# Competitor price: fill with Selling_Price * random factor if missing
df['Competitor_Price'] = df['Competitor_Price'].fillna((df['Selling_Price'] * np.random.uniform(0.95,1.05, size=len(df))).round(2))

# Profit & margin
df['Gross_Profit_per_unit'] = (df['Selling_Price'] - df['Cost']).round(2)
df['Gross_Profit_Total'] = (df['Gross_Profit_per_unit'] * df['Units_Sold']).round(2)
df['Margin_%'] = ((df['Gross_Profit_per_unit'] / df['Selling_Price']) * 100).round(2)

# Price delta vs competitor
df['Price_Delta_vs_Competitor'] = (df['Selling_Price'] - df['Competitor_Price']).round(2)

# Simple elasticity proxy: correlation of price and units sold in small rolling groups (approx)
df['Price_Elasticity_Score'] = df.groupby(['Product_ID'])['Selling_Price'].transform(
    lambda x: np.round(np.abs(np.polyfit(np.arange(len(x)), x.fillna(method='ffill'), 1)[0]) if len(x.dropna())>1 else 0, 4)
)

# Date components for Power BI
df['Month'] = df['Date'].apply(lambda d: d.strftime("%b"))
df['Month_Number'] = df['Date'].apply(lambda d: d.month)
df['Year'] = df['Date'].apply(lambda d: d.year)
df['MonthYear'] = df['Date'].apply(lambda d: d.strftime("%b %Y"))

# Final cleaned file
raw_path = os.path.join(FOLDER, "dataset", "retail_pricing_raw.csv")
clean_path = os.path.join(FOLDER, "output", "retail_pricing_cleaned.csv")
df.to_csv(raw_path, index=False)
df.to_csv(clean_path, index=False)

print("Files written:")
print(raw_path)
print(clean_path)


C:\Users\sxganesan\AppData\Local\Temp\ipykernel_15100\1240731117.py:92: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  lambda x: np.round(np.abs(np.polyfit(np.arange(len(x)), x.fillna(method='ffill'), 1)[0]) if len(x.dropna())>1 else 0, 4)


Files written:
retail-pricing-analysis\dataset\retail_pricing_raw.csv
retail-pricing-analysis\output\retail_pricing_cleaned.csv
